In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

This notebook is a revised version of notebook from [Amy Wu](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/matching_engine/intro-swivel.ipynb)

# E2E ML on GCP: MLOps stage 6 : serving: get started with Vertex AI Matching Engine and Swivel builtin algorithm

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_matching_engine_swivel.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_matching_engine_swivel.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_matching_engine_swivel.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

This notebook demonstrate how to train an embedding with Submatrix-wise Vector Embedding Learner ([Swivel](https://arxiv.org/abs/1602.02215)) using Vertex AI Pipelines. The purpose of the embedding learner is to compute cooccurrences between tokens in a given dataset and to use the cooccurrences to generate embeddings.

Vertex AI provides a pipeline template for training with Swivel, so you don't need to design your own pipeline or write
your own training code.

### Objective

In this notebook, you learn how to train custom embeddings using Vertex AI Pipelines and subsequently train and deploy a matching engine index using the embeddings.

This tutorial uses the following Google Cloud ML services:

- `Vertex AI Swivel` builtin algorithm
- `Vertex AI Matching Engine`
- `Vertex AI Batch Prediction`

The steps performed include:

1. Train the `Swivel` algorithm to generate embeddings (encoder) for the dataset.
2. Make example predictions (embeddings) from then trained encoder.
3. Generate embeddings using the trained `Swivel` builtin algorithm.
4. Store embeddings to format supported by `Matching Engine`.
5. Create a `Matching Engine Index` for the embeddings.
6. Deploy the `Matching Engine Index` to a `Index Endpoint`.
7. Make a matching engine prediction request.

### Dataset

This tutorial uses the `movielens sample dataset` in the public bucket `gs://cloud-samples-data/vertex-ai/matching-engine/swivel`, which was generated from the [MovieLens movie rating dataset](https://grouplens.org/datasets/movielens/100k/). This dataset is processed so that each line contains the movies that have same rating by the same user. The directory also includes `movies.csv`, which maps the movie ids to their names.

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Dataflow
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and [Dataflow pricing](https://cloud.google.com/dataflow/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Vertex AI Workbench Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

## Installation

Install packages required for executing this notebook.

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install {USER_FLAG} --upgrade pip -q
! pip3 install {USER_FLAG} --upgrade scikit-learn -q
! pip3 install {USER_FLAG} --upgrade google-cloud-aiplatform tensorboard-plugin-profile -q
! pip3 install {USER_FLAG} --upgrade tensorflow -q

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Dataflow API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,dataflow.googleapis.com).

1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Get your project number

Now that the project ID is set, you get your corresponding project number.

In [ ]:
shell_output = ! gcloud projects list --filter="PROJECT_ID:'{PROJECT_ID}'" --format='value(PROJECT_NUMBER)'
PROJECT_NUMBER = shell_output[0]
print("Project Number:", PROJECT_NUMBER)

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebooks**, your environment is already
authenticated. 

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you submit a built-in Swivel job using the Cloud SDK, you need a Cloud Storage bucket for storing the input dataset and pipeline artifacts (the trained model).

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_URI

### Service Account

**If you don't know your service account**, try to get your service account using gcloud command by executing the second cell below.

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

#### Set service account access for Vertex AI Pipelines

Run the following commands to grant your service account access to read and write pipeline artifacts in the bucket that you created in the previous step -- you only need to run these once per service account.

In [ ]:
!gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_NAME

!gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_NAME

### Import libraries and define constants
Define constants used in this tutorial.

In [ ]:
SOURCE_DATA_PATH = "{}/swivel".format(BUCKET_URI)
PIPELINE_ROOT = "{}/pipeline_root".format(BUCKET_URI)

Import packages used in this tutorial.

In [ ]:
import json

import pandas as pd
import tensorflow as tf
from google.cloud import aiplatform
from sklearn.metrics.pairwise import cosine_similarity

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

#### Set machine type

Next, set the machine type to use for prediction.

- Set the variable `DEPLOY_COMPUTE` to configure  the compute resources for the VMs you will use for for prediction.
 - `machine type`
     - `n1-standard`: 3.75GB of memory per vCPU.
     - `n1-highmem`: 6.5GB of memory per vCPU
     - `n1-highcpu`: 0.9 GB of memory per vCPU
 - `vCPUs`: number of \[2, 4, 8, 16, 32, 64, 96 \]

*Note: You may also use n2 and e2 machine types for training and deployment, but they do not support GPUs*.

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

### Copy the Swivel template

First, download the provioded Swivel template and configuration script.

In [ ]:
! gsutil cp gs://cloud-samples-data/vertex-ai/matching-engine/swivel/pipeline/* .

### Set your pipeline configurations

Change your pipeline configurations: 

* pipeline_suffix: Suffix of your pipeline name (lowercase and hyphen are allowed).
* machine_type: e.g. n1-standard-16.
* accelerator_count: Number of GPUs in each machine.
* accelerator_type: e.g. NVIDIA_TESLA_P100, NVIDIA_TESLA_V100.
* region: e.g. us-east1 (optional, default is us-central1)
* network_name: e.g., my_network_name (optional, otherwise it uses "default" network).

### VPC Network peering, subnetwork and private IP address configuration

Executing the following cell will generate two files:
1. `swivel_pipeline_basic.json`: The basic template allows public IPs and default network for the Dataflow job, and doesn't require setting up VPC Network peering for Vertex AI and **you will use it in this notebook sample**.
1. `swivel_pipeline.json`: This template enables private IPs and subnet configuration for the Dataflow job, also requires setting up VPC Network peering for the Vertex custom training. This template includes the following args:
* "--subnetwork=regions/%REGION%/subnetworks/%NETWORK_NAME%",
* "--no_use_public_ips",
* \"network\": \"projects/%PROJECT_NUMBER%/global/networks/%NETWORK_NAME%\"

**WARNING** In order to specify private IPs and configure VPC network, you need to [set up VPC Network peering for Vertex AI](https://cloud.google.com/vertex-ai/docs/general/vpc-peering#overview) for your subnetwork (e.g. "default" network on "us-central1") before submitting the following job. This is required for using private IP addresses for DataFlow and Vertex AI.

In [ ]:
YOUR_PIPELINE_SUFFIX = "swivel-pipeline-movie"  # @param {type:"string"}
MACHINE_TYPE = "n1-standard-16"  # @param {type:"string"}
ACCELERATOR_COUNT = 2  # @param {type:"integer"}
ACCELERATOR_TYPE = "NVIDIA_TESLA_V100"  # @param {type:"string"}

! chmod +x swivel_template_configuration*

! ./swivel_template_configuration_basic.sh -pipeline_suffix {YOUR_PIPELINE_SUFFIX} -project_number {PROJECT_NUMBER} -project_id {PROJECT_ID} -machine_type {MACHINE_TYPE} -accelerator_count {ACCELERATOR_COUNT} -accelerator_type {ACCELERATOR_TYPE} -pipeline_root {BUCKET_NAME}
! ./swivel_template_configuration.sh -pipeline_suffix {YOUR_PIPELINE_SUFFIX} -project_number {PROJECT_NUMBER} -project_id {PROJECT_ID} -machine_type {MACHINE_TYPE} -accelerator_count {ACCELERATOR_COUNT} -accelerator_type {ACCELERATOR_TYPE} -pipeline_root {BUCKET_NAME}

! sed "s/\\t/    /g" swivel_pipeline_basic.json > tmp.json
! mv tmp.json swivel_pipeline_basic.json

Both `swivel_pipeline_basic.json` and `swivel_pipeline.json` are generated.

## Create the Swivel job for MovieLens items embeddings

You will submit the pipeline job by passing the compiled specification to the `create_run_from_job_spec()` method. Note that you are passing a `parameter_values` dictionary that specifies the pipeline input parameters to use.

The following table shows the runtime parameters required by the Swivel job:

| Parameter                  |Data type | Description                                                        | Required               |
|----------------------------|----------|--------------------------------------------------------------------|------------------------|
| `embedding_dim`            | int      | Dimensions of the embeddings to train.                         | No - Default is 100    |
| `input_base`         | string   | Cloud Storage path where the input data is stored.                       | Yes                    |
| `input_type`               | string   | Type of the input data.  Can be either 'text' (for wikipedia sample) or 'items'(for movielens sample).      | Yes                    |
| `max_vocab_size`               | int      | Maximum vocabulary size to generate embeddings for.                | No - Default is 409600 |
|`num_epochs` | int | Number of epochs for training. | No - Default is 20 |

In short, the **items** input type means that each line of your input data should be space-separated item ids. Each line is tokenized by splitting on whitespace. The **text** input type means that each line of your input data should be equivalent to a sentence. Each line is tokenized by lowercasing, and splitting on whitespace.


In [ ]:
# MovieLens items embedding sample

PARAMETER_VALUES = {
    "embedding_dim": 100,  # <---CHANGE THIS (OPTIONAL)
    "input_base": "{}/movielens_25m/train".format(SOURCE_DATA_PATH),
    "input_type": "items",  # For movielens sample
    "max_vocab_size": 409600,  # <---CHANGE THIS (OPTIONAL)
    "num_epochs": 5,  # <---CHANGE THIS (OPTIONAL)
}

#### Copy the dataset to Cloud Storage

Next, copy the MovieLens dataset to your Cloud Storage bucket.

In [ ]:
# Copy the MovieLens sample dataset
! gsutil cp -r gs://cloud-samples-data/vertex-ai/matching-engine/swivel/movielens_25m/train/* {SOURCE_DATA_PATH}/movielens_25m

### Submit the pipeline job

Next, submit the pipeline job to `Vertex AI Pipelines`.

In [ ]:
# Instantiate PipelineJob object
pl = aiplatform.PipelineJob(
    display_name=YOUR_PIPELINE_SUFFIX,
    # Whether or not to enable caching
    # True = always cache pipeline step result
    # False = never cache pipeline step result
    # None = defer to cache option for each pipeline component in the pipeline definition
    enable_caching=False,
    # Local or GCS path to a compiled pipeline definition
    template_path="swivel_pipeline_basic.json",
    # Dictionary containing input parameters for your pipeline
    parameter_values=PARAMETER_VALUES,
    # GCS path to act as the pipeline root
    pipeline_root=PIPELINE_ROOT,
)

# Submit the Pipeline to Vertex AI
# Optionally you may specify the service account below: submit(service_account=SERVICE_ACCOUNT)
# You must have iam.serviceAccounts.actAs permission on the service account to use it
pl.submit()

After the job is submitted successfully, you can view its details (including run name that you'll need below) and logs.

### Use TensorBoard to check the model

You may use the TensorBoard to check the model training process. In order to do that, you need to find the path to the trained model artifact. After the job finishes successfully (~ a few hours), you can view the trained model output path in the [Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata/introduction) browser. It is going to have the following format:

* {BUCKET_URI}/pipeline_root/{PROJECT_NUMBER}/swivel-{TIMESTAMP}/EmbTrainerComponent_-{SOME_NUMBER}/model/

You may copy this path for the MODELOUTPUT_DIR below.

Alternatively, you can download a pretrained model to `{SOURCE_DATA_PATH}/movielens_model` and proceed. This pretrained model is for demo purpose and not optimized for production usage.

In [ ]:
! gsutil -m cp -r gs://cloud-samples-data/vertex-ai/matching-engine/swivel/models/movielens/model {SOURCE_DATA_PATH}/movielens_model

In [ ]:
SAVEDMODEL_DIR = os.path.join(SOURCE_DATA_PATH, "movielens_model/model")
LOGS_DIR = os.path.join(SOURCE_DATA_PATH, "movielens_model/tensorboard")

When the training starts, you can view the logs in TensorBoard:

In [ ]:
# If on Vertex AI Workbench Notebooks, then don't execute this code.
if not os.getenv("DL_ANACONDA_HOME"):
    if "google.colab" in sys.modules:
        # Load the TensorBoard notebook extension.
        %load_ext tensorboard

In [ ]:
# If on Vertex AI Workbench Notebooks, then don't execute this code.
if not os.getenv("DL_ANACONDA_HOME"):
    if "google.colab" in sys.modules:
        %tensorboard --logdir $LOGS_DIR

For **Vertex AI Workbench Notebooks**, you can do the following:

1. Open Cloud Shell from the Google Cloud Console.
2. Install dependencies: `pip3 install tensorflow tensorboard-plugin-profile`
3. Run the following command: `tensorboard --logdir {LOGS_DIR}`. You will see a message "TensorBoard 2.x.0 at http://localhost:<PORT>/ (Press CTRL+C to quit)" as the output. Take note of the port number.
4. You can click on the Web Preview button and view the TensorBoard dashboard and profiling results. You need to configure Web Preview's port to be the same port as you receive from step 3.

### Upload the model to `Vertex AI Model` resource

First, import the model using the `upload()` method, with the following parameters:

- `display_name`: A human readable name for the model resource.
- `artifact_uri`: The Cloud Storage location of the model artifacts.
- `serving_container_image_uri`: The deployment container. In this tutorial, you use the prebuilt Two-Tower deployment container.

In [ ]:
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-4:latest"

# Upload the trained model to Model resource
model = aiplatform.Model.upload(
    display_name="movies_" + TIMESTAMP,
    artifact_uri=SAVEDMODEL_DIR,
    serving_container_image_uri=DELOY_IMAGE,
)

## Deploy the model to `Vertex AI Endpoint`

Deploying the `Vertex AI Model` resoure to a `Vertex AI Endpoint` for online predictions:

1. Create an `Endpoint` resource exposing an external interface to users consuming the model. 
2. After the `Endpoint` is ready, deploy one or more instances of a model to the `Endpoint`. The deployed model runs the Swivel encoder to serve embeddings.

Refer to Vertex AI Predictions guide to [Deploy a model using the Vertex AI API](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api) for more information about the APIs used in the following cells.

### Create a `Vertex AI Endpoint`

Next, you create the `Vertex AI Endpoint`, from which you subsequently deploy your `Vertex AI Model` resource to.

In [ ]:
endpoint = aiplatform.Endpoint.create(display_name="swivel_embedding_" + TIMESTAMP)

### Deploying `Model` resources to an `Endpoint` resource.

You can deploy one of more `Vertex AI Model` resource instances to the same endpoint. Each `Vertex AI Model` resource that is deployed will have its own deployment container for the serving binary. 

In the next example, you deploy the `Vertex AI Model` resource to a `Vertex AI Endpoint` resource. The `Vertex AI Model` resource already has defined for it the deployment container image. To deploy, you specify the following additional configuration settings:

- The machine type.
- The (if any) type and number of GPUs.
- Static, manual or auto-scaling of VM instances.

In this example, you deploy the model with the minimal amount of specified parameters, as follows:

- `model`: The `Model` resource.
- `deployed_model_displayed_name`: The human readable name for the deployed model instance.
- `machine_type`: The machine type for each VM instance.

Do to the requirements to provision the resource, this may take upto a few minutes.

In [ ]:
response = endpoint.deploy(
    model=model,
    deployed_model_display_name=DISPLAY_NAME,
    machine_type=DEPLOY_COMPUTE,
    traffic_split={"0": 100},
)

print(endpoint)

### Load the movie ids and titles for querying embeddings

Next, download the movie IDs and titles sample dataset and read them into a pandas Dataframe.

In [ ]:
! gsutil cp gs://cloud-samples-data/vertex-ai/matching-engine/swivel/movielens_25m/movies.csv ./movies.csv

movies = pd.read_csv("movies.csv")
print(f"Movie count: {len(movies.index)}")
movies.head()

## Creating embeddings

Now that you have deployed the encoder model on `Vertex AI Prediction`, you can call the model to generate embeddings for new data. 

### Make an online prediction with SDK

[Online prediction](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models) is used to synchronously query a model on a small batch of instances with minimal latency. The following function calls the deployed model using Vertex AI SDK for Python.

BLAH The input data you want predicted embeddings on should be provided as a list of movie IDs. Note that you should also provide a unique `key` field (of type str) for each input instance so that you can associate each output embedding with its corresponding input.

In [ ]:
# Change to your favourite movies.
query_movies = [
    "Lion King, The (1994)",
    "Aladdin (1992)",
    "Star Wars: Episode IV - A New Hope (1977)",
    "Star Wars: Episode VI - Return of the Jedi (1983)",
    "Terminator 2: Judgment Day (1991)",
    "Aliens (1986)",
    "Godfather, The (1972)",
    "Goodfellas (1990)",
]


def get_movie_id(title):
    return list(movies[movies.title == title].movieId)[0]


instances = [str(get_movie_id(title)) for title in query_movies]
print(instances)

### Make an online prediction request for embeddings

Next, make the prediction request using the `predict()` method.

In [ ]:
predictions = endpoint.predict(instances=instances)
embeddings = predictions.predictions
print("Number of embeddings:", len(embeddings))
print(embeddings[0])

#### Explore the movie embedding

Using the embeddings, explore how similiar each movie in the sample movie list is similiar to each other movie in the sample movie list.

*Note:* A value of 1.0 means they are the same embedding.

In [ ]:
for idx1 in range(0, len(input_items) - 1, 2):
    item1 = instances[idx1]
    title1 = query_movies[idx1]
    print(title1)
    print("==================")
    embedding1 = embeddings[idx1]
    for idx2 in range(0, len(instances)):
        item2 = input_items[idx2]
        embedding2 = embeddings[idx2]
        similarity = round(cosine_similarity([embedding1], [embedding2])[0][0], 5)
        title1 = query_movies[idx1]
        title2 = query_movies[idx2]
        print(f" - Similarity to '{title2}' = {similarity}")
    print()

You can use the [TensorBoard Embedding Projector](https://www.tensorflow.org/tensorboard/tensorboard_projector_plugin) to graphically represent high dimensional embeddings, which can be helpful in examining and understanding your embeddings.

### Make an online prediction with `gcloud`

You can also do online prediction using the gcloud CLI.

In [ ]:
import json

request = json.dumps({"instances": input_items})
with open("request.json", "w") as writer:
    writer.write(f"{request}\n")

ENDPOINT_ID = endpoint.resource_name

! gcloud ai endpoints predict {ENDPOINT_ID} \
  --region={REGION} \
  --json-request=request.json

### Make a batch prediction

[Batch prediction](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions) is used to asynchronously make predictions on a batch of input data.  This is recommended if you have a large input size and do not need an immediate response, such as getting embeddings for candidate objects in order to create an index for a nearest neighbor search service such as [Vertex Matching Engine](https://cloud.google.com/vertex-ai/docs/matching-engine/overview).

### Create the batch input file

Next, you generate the batch input file to generate embeddings for the dataset, which you subsequently use to create an index with `Vertex AI Matching Engine`. In this example, the dataset contains a 200000 unique identifiers (1...200000). You will use the trained encoder to generate a predicted embedding for each unique identifier.

The input data needs to be on Cloud Storage and in JSONL format. You can use the sample query object file provided below. Like with online prediction, it's recommended to have the `key` field so that you can associate each output embedding with its corresponding input.

In [ ]:
QUERY_EMBEDDING_PATH = f"{BUCKET_URI}/embeddings/train.jsonl"

import tensorflow as tf

with tf.io.gfile.GFile(QUERY_EMBEDDING_PATH, "w") as f:
    for i in range(1, 200001):
        query = str(i)
        f.write(json.dumps(query) + "\n")

print("\nNumber of embeddings: ")
! gsutil cat {QUERY_EMBEDDING_PATH} | wc -l

In [ ]:
! gsutil cat {QUERY_EMBEDDING_PATH} | head

### Send the prediction request

To make a batch prediction request, call the model object's `batch_predict` method with the following parameters: 
- `instances_format`: The format of the batch prediction request file: "jsonl", "csv", "bigquery", "tf-record", "tf-record-gzip" or "file-list"
- `prediction_format`: The format of the batch prediction response file: "jsonl", "csv", "bigquery", "tf-record", "tf-record-gzip" or "file-list"
- `job_display_name`: The human readable name for the prediction job.
- `gcs_source`: A list of one or more Cloud Storage paths to your batch prediction requests.
- `gcs_destination_prefix`: The Cloud Storage path that the service will write the predictions to.
- `model_parameters`: Additional filtering parameters for serving prediction results.
- `machine_type`: The type of machine to use for training.
- `accelerator_type`: The hardware accelerator type.
- `accelerator_count`: The number of accelerators to attach to a worker replica.
- `starting_replica_count`: The number of compute instances to initially provision.
- `max_replica_count`: The maximum number of compute instances to scale to. In this tutorial, only one instance is provisioned.

### Compute instance scaling

You can specify a single instance (or node) to process your batch prediction request. This tutorial uses a single node, so the variables `MIN_NODES` and `MAX_NODES` are both set to `1`.

If you want to use multiple nodes to process your batch prediction request, set `MAX_NODES` to the maximum number of nodes you want to use. Vertex AI autoscales the number of nodes used to serve your predictions, up to the maximum number you set. Refer to the [pricing page](https://cloud.google.com/vertex-ai/pricing#prediction-prices) to understand the costs of autoscaling with multiple nodes.


In [ ]:
MIN_NODES = 1
MAX_NODES = 4

batch_predict_job = model.batch_predict(
    job_display_name=f"batch_predict_swivel",
    gcs_source=[QUERY_EMBEDDING_PATH],
    gcs_destination_prefix=f"{BUCKET_URI}/embeddings/output",
    machine_type=DEPLOY_COMPUTE,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
)

### Get the predicted embeddings

Next, get the results from the completed batch prediction job.

The results are written to the Cloud Storage output bucket you specified in the batch prediction request. You call the method `iter_outputs()` to get a list of each Cloud Storage file generated with the results. Each file contains one or more prediction requests in a JSON format:

- `instance`: The prediction request.
- `prediction`: The prediction response.

In [ ]:
bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

result_files = []
for prediction_result in prediction_results:
    result_file = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    result_files.append(result_file)

print(result_files)

#### Save the embeddings in JSONL format

Next, you store the predicted embeddings as a JSONL formatted file. Each embedding is stored as:

    { 'id': .., 'embedding': [ ... ] }
    
The format of the embeddings for the index can be in either CSV, JSON, or Avro format.

Learn more about [Embedding Formats for Indexing](https://cloud.google.com/vertex-ai/docs/matching-engine/using-matching-engine#json)

In [ ]:
embeddings = []
for result_file in result_files:

    with tf.io.gfile.GFile(result_file, "r") as f:
        instances = list(f)

    for instance in instances:
        instance = instance.replace('\\"', "'")
        result = json.loads(instance)
        prediction = result["prediction"]
        key = result["instance"]

        embedding = {"id": key, "embedding": prediction}
        embeddings.append(embedding)

print("Number of embeddings", len(embeddings))
print("Encoding Dimensions", len(embeddings[0]["embedding"]))
print("Example embedding", embeddings[0])

with open("embeddings.json", "w") as f:
    for i in range(len(embeddings)):
        f.write(json.dumps(embeddings[i]).replace('"', "'"))
        f.write("\n")

! head -n 2 embeddings.json

#### Store the JSONL formatted embeddings in Cloud Storage

Next, you upload the training data to your Cloud Storage bucket.

In [ ]:
EMBEDDINGS_URI = f"{BUCKET_URI}/embeddings/swivel/"
! gsutil cp embeddings.json {EMBEDDINGS_URI}

### Create Matching Engine Index

Next, you create the index for your embeddings. Currently, two indexing algorithms are supported:

- `create_tree_ah_index()`:  Shallow tree + Asymmetric hashing.
- `create_brute_force_index()`: Linear search.

In this tutorial, you use the `create_tree_ah_index()`for production scale. The method is called with the following parameters:

- `display_name`: A human readable name for the index.
- `contents_delta_uri`: A Cloud Storage location for the embeddings, which are either to be inserted, updated or deleted.
- `dimensions`: The number of dimensions of the input vector
- `approximate_neighbors_count`: (for Tree AH) The default number of neighbors to find via approximate search before exact reordering is performed. Exact reordering is a procedure where results returned by an approximate search algorithm are reordered via a more expensive distance computation.
- `distance_measure_type`: The distance measure used in nearest neighbor search.
    - `SQUARED_L2_DISTANCE`: Euclidean (L2) Distance
    - `L1_DISTANCE`: Manhattan (L1) Distance
    - `COSINE_DISTANCE`: Cosine Distance. Defined as 1 - cosine similarity.
    - `DOT_PRODUCT_DISTANCE`: Default value. Defined as a negative of the dot product.
- `description`: A human readble description of the index.
- `labels`: User metadata in the form of a dictionary.
- `leaf_node_embedding_count`: Number of embeddings on each leaf node. The default value is 1000 if not set.
- `leaf_nodes_to_search_percent`: The default percentage of leaf nodes that any query may be searched. Must be in range 1-100, inclusive. The default value is 10 (means 10%) if not set.

This may take upto 30 minutes.

Learn more about [Configuring Matching Engine Indexes](https://cloud.google.com/vertex-ai/docs/matching-engine/configuring-indexes).

In [ ]:
DIMENSIONS = len(embeddings[0]["embedding"])
DISPLAY_NAME = "movies"

tree_ah_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=DISPLAY_NAME,
    contents_delta_uri=EMBEDDINGS_URI,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=50,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    description="Two tower generated embeddings",
    labels={"label_name": "label_value"},
    # TreeAH specific parameters
    leaf_node_embedding_count=100,
    leaf_nodes_to_search_percent=7,
)

INDEX_RESOURCE_NAME = tree_ah_index.resource_name
print(INDEX_RESOURCE_NAME)

## Setup VPC peering network

To use a `Matching Engine Index`, you setup a VPC peering network between your project and the `Vertex AI Matching Engine` service project. This eliminates additional hops in network traffic and allows using efficient gRPC protocol.

Learn more about [VPC peering](https://cloud.google.com/vertex-ai/docs/general/vpc-peering).

**IMPORTANT: you can only setup one VPC peering to servicenetworking.googleapis.com per project.**

### Create VPC peering for default network

For simplicity, we setup VPC peering to the default network. You can create a different network for your project.

If you setup VPC peering with any other network, make sure that the network already exists and that your VM is running on that network.

In [ ]:
# This is for display only; you can name the range anything.
PEERING_RANGE_NAME = "vertex-ai-prediction-peering-range"
NETWORK = "default"

# NOTE: `prefix-length=16` means a CIDR block with mask /16 will be
# reserved for use by Google services, such as Vertex AI.
! gcloud compute addresses create $PEERING_RANGE_NAME \
  --global \
  --prefix-length=16 \
  --description="peering range for Google service" \
  --network=$NETWORK \
  --purpose=VPC_PEERING

### Create the VPC connection

Next, create the connection for VPC peering.

*Note:* If you get a PERMISSION DENIED, you may not have the neccessary role 'Compute Network Admin' set for your default service account. In the Cloud Console, do the following steps.

1. Goto `IAM & Admin`
2. Find your service account.
3. Click edit icon.
4. Select `Add Another Role`.
5. Enter 'Compute Network Admin'.
6. Select `Save`

In [ ]:
! gcloud services vpc-peerings connect \
  --service=servicenetworking.googleapis.com \
  --network=$NETWORK \
  --ranges=$PEERING_RANGE_NAME \
  --project=$PROJECT_ID

Check the status of your peering connections.

In [ ]:
! gcloud compute networks peerings list --network $NETWORK

#### Construct the full network name

You need to have the full network resource name when you subsequently create an `Matching Engine Index Endpoint` resource for VPC peering.

In [ ]:
full_network_name = f"projects/{PROJECT_NUMBER}/global/networks/{NETWORK}"

### Create an IndexEndpoint with VPC Network

Next, you create a `Matching Engine Index Endpoint`, similar to the concept of creating a `Private Endpoint` for prediction with a peer-to-peer network.

To create the `Index Endpoint` resource, you call the method `create()` with the following parameters:

- `display_name`: A human readable name for the `Index Endpoint`.
- `description`: A description for the `Index Endpoint`.
- `network`: The VPC network resource name.

In [ ]:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name="index_endpoint_for_demo",
    description="index endpoint description",
    network=full_network_name,
)

INDEX_ENDPOINT_NAME = index_endpoint.resource_name
print(INDEX_ENDPOINT_NAME)

### Deploy the `Matching Engine Index` to the `Index Endpoint` resource

Next, deploy your index to the `Index Endpoint` using the method `deploy_index()` with the following parameters:

- `display_name`: A human readable name for the deployed index.
- `index`: Your index.
- `deployed_index_id`: A user assigned identifier for the deployed index.
- `machine_type`: (optional) The VM instance type.
- `min_replica_count`: (optional) Minimum number of VM instances for auto-scaling.
- `max_replica_count`: (optional) Maximum number of VM instances for auto-scaling.

Learn more about [Machine resources for Index Endpoint](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.indexEndpoints#DeployedIndex)

In [ ]:
DEPLOYED_INDEX_ID = "tree_ah_twotower_deployed_" + TIMESTAMP

MIN_NODES = 1
MAX_NODES = 2
DEPLOY_COMPUTE = "n1-standard-16"

index_endpoint.deploy_index(
    display_name="deployed_index_for_demo",
    index=tree_ah_index,
    deployed_index_id=DEPLOYED_INDEX_ID,
    machine_type=DEPLOY_COMPUTE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
)

print(index_endpoint.deployed_indexes)

### Create and execute an online query

Now that your index is deployed, you can make queries.

First, you construct a vector `query` using synthetic data, to use as the example to return matches for.

Next, you make the matching request using the method `match()`, with the following parameters:

- `deployed_index_id`:  The identifier of the deployed index.
- `queries`: A list of queries (instances).
- `num_neighbors`: The number of closest matches to return.

In [ ]:
# The number of nearest neighbors to be retrieved from database for each query.
NUM_NEIGHBOURS = 10

# Test query
queries = [embeddings[0]["embedding"], embeddings[1]["embedding"]]

matches = index_endpoint.match(
    deployed_index_id=DEPLOYED_INDEX_ID, queries=queries, num_neighbors=NUM_NEIGHBOURS
)

for instance in matches:
    print("INSTANCE")
    for match in instance:
        print(match)

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:


In [ ]:
# Delete endpoint resource
endpoint.delete(force=True)

# Delete model resource
model.delete()

# Force undeployment of indexes and delete endpoint
try:
    index_endpoint.delete(force=True)
except Exception as e:
    print(e)

# Delete indexes
try:
    tree_ah_index.delete()
except Exception as e:
    print(e)

# Delete Cloud Storage objects that were created
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $OUTPUT_DIR